In [1]:
import pandas as pd
import numpy as np

In [3]:
Card_spend=pd.read_csv("./01_제공데이터_update(210811)/04_음식관련 카드소비_CARD_SPENDING.CSV", encoding='cp949')

In [4]:
Card_spend.head(10)

,base_date,city,emd_cd,emd_nm,mct_cat_cd,mct_cat_nm,use_cnt,use_amt
0,2018-01-06,제주시,50110650,연동,C00100,한식,3643,127777300
1,2018-01-09,제주시,50110650,연동,C00500,패스트푸드,432,6711675
2,2018-01-15,제주시,50110650,연동,C01200,농축수산물,236,16089579
3,2018-01-15,제주시,50110650,연동,A00200,마트/슈퍼마켓,4031,121979867
4,2018-01-20,제주시,50110650,연동,C01000,식품,633,30410674
5,2018-01-22,제주시,50110650,연동,C00900,주점및주류판매,49,2212446
6,2018-01-22,제주시,50110650,연동,C00200,양식,98,2912705
7,2018-01-26,제주시,50110650,연동,C00200,양식,182,7191382
8,2018-01-30,제주시,50110650,연동,C01000,식품,481,18497443
9,2018-01-31,제주시,50110650,연동,C00500,패스트푸드,555,8944547


In [23]:
# 업종 확인
업종=Card_spend['mct_cat_nm'].unique()
len(업종)

# [가정음쓰]
# 배달 C01400
# 간식 C00400
# 식품 C01000
# 마트/슈퍼마켓 A00200
# 농축수산물 C01200

# [식당음쓰]
# 한식 C00100
# 패스트푸드 C00500
# 주점 및 주류 판매 C00900
# 양식 C00200
# 부페 C00700
# 아시아음식 C00300

11

In [57]:
# 필요없는 열(날짜, 행정동코드, 행정동명, 업종명) 삭제
Card_spend_2=Card_spend.drop(['city','emd_nm','mct_cat_nm'], 1)
Card_spend_2

,base_date,emd_cd,mct_cat_cd,use_cnt,use_amt
0,2018-01-06,50110650,C00100,3643,127777300
1,2018-01-09,50110650,C00500,432,6711675
2,2018-01-15,50110650,C01200,236,16089579
3,2018-01-15,50110650,A00200,4031,121979867
4,2018-01-20,50110650,C01000,633,30410674
...,...,...,...,...,...
544800,2020-10-16,XXXXXXXX,C01400,5,127250
544801,2019-08-31,XXXXXXXX,C01400,5,39974
544802,2020-10-03,XXXXXXXX,C01400,10,329832
544803,2020-10-23,XXXXXXXX,C01400,5,68715


In [69]:
# 날짜데이터 datetime으로 변환
Card_spend_2['base_date'] = pd.to_datetime(Card_spend_2['base_date'], format="%Y-%m-%d")
Card_spend_2.set_index(Card_spend_2['base_date'])

# "알수없음" 삭제
idx = Card_spend_2[Card_spend_2['emd_cd'] == "XXXXXXXX"].index
Card_spend_2 = Card_spend_2.drop(idx)

# 행정동코드 int형으로 변환
Card_spend_2['emd_cd'] = pd.to_numeric(Card_spend_2['emd_cd'])


In [122]:
home_cd=['C01400','C00400','C01000','A00200','C01200'] # home음쓰
out_cd=['C00100','C00500','C00900','C00200','C00700','C00300'] # out음쓰

Card_spend_2=pd.DataFrame(Card_spend_2,columns=['base_date', 'emd_cd', 'mct_cat_cd', 'use_cnt', 'use_amt'])

# Card_spend_out(외부음쓰) : home_cd 행 삭제

home = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C01400'].index
Card_spend_out0 = Card_spend_2.drop(home)

home = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C00400'].index
Card_spend_out1 = Card_spend_out0.drop(home)

home = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C01000'].index
Card_spend_out2 = Card_spend_out1.drop(home)

home = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'A00200'].index
Card_spend_out3 = Card_spend_out2.drop(home)

home = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C01200'].index
Card_spend_out = Card_spend_out3.drop(home)


Card_spend_out['mct_cat_cd'].unique()
Card_spend_out # 외부 음쓰만 분류된 df

,base_date,emd_cd,mct_cat_cd,use_cnt,use_amt
0,2018-01-06,50110650,C00100,3643,127777300
1,2018-01-09,50110650,C00500,432,6711675
5,2018-01-22,50110650,C00900,49,2212446
6,2018-01-22,50110650,C00200,98,2912705
7,2018-01-26,50110650,C00200,182,7191382
...,...,...,...,...,...
533744,2021-04-27,50110520,C00900,56,1953471
533746,2020-05-09,50110520,C00200,143,4150853
533748,2021-04-01,50110520,C00300,162,5163795
533749,2018-09-03,50110520,C00700,5,68740


In [123]:
# Card_spend_home(가정음쓰) = out_cd 행 삭제
out_cd=['C00100','C00500','C00900','C00200','C00700','C00300']


out = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C00100'].index
Card_spend_home0 = Card_spend_2.drop(out)

out = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C00500'].index
Card_spend_home1 = Card_spend_home0.drop(out)

out = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C00900'].index
Card_spend_home2 = Card_spend_home1.drop(out)

out = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C00200'].index
Card_spend_home3 = Card_spend_home2.drop(out)

out = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C00700'].index
Card_spend_home4 = Card_spend_home3.drop(out)

out = Card_spend_2[Card_spend_2['mct_cat_cd'] == 'C00300'].index
Card_spend_home = Card_spend_home4.drop(out)

Card_spend_home['mct_cat_cd'].unique()
Card_spend_home # 가정 음쓰만 분류된 df

,base_date,emd_cd,mct_cat_cd,use_cnt,use_amt
2,2018-01-15,50110650,C01200,236,16089579
3,2018-01-15,50110650,A00200,4031,121979867
4,2018-01-20,50110650,C01000,633,30410674
8,2018-01-30,50110650,C01000,481,18497443
10,2018-02-01,50110650,C00400,846,10212423
...,...,...,...,...,...
544725,2020-05-26,50110520,C01400,445,10786980
544726,2020-12-08,50110520,C01400,513,11768033
544727,2020-04-30,50110520,C01400,486,11109119
544728,2020-05-28,50110520,C01400,353,8428342


In [139]:
# 행정동코드 매개변수로 받아서 해당 구역의 '가정 배출량'을 월별로 정리

pd.options.display.float_format = '{:.5f}'.format
def monthly_output(emdcd):
    emd_data = Card_spend_home[Card_spend_home['emd_cd'] == emdcd]    # 해당 행정동 데이터만 추출
    emd_data_monthly = emd_data.resample('M', on='base_date').mean()    # 월별로 합산
    emd_data_monthly['emd_cd'] = emdcd
    return emd_data_monthly

In [140]:
emd_cd_ary = [50110250, 50110253, 50110256, 50110259, 50110310, 50110320, 50110330, 50110510, 50110520, 50110530, 50110540, 50110550, 50110560, 50110570, 50110580, 50110590, 50110600, 50110610, 50110620, 50110630, 50110640, 50110650, 50110660, 50110670, 50110680, 50110690, 50130250, 50130253, 50130259, 50130310, 50130320, 50130510, 50130520, 50130530, 50130540, 50130550,
              50130560, 50130570, 50130580, 50130590, 50130600, 50130610, 50130620]

data_result=pd.DataFrame(index=range(0,0), columns=['base_date','emd_cd','mct_cat_cd','use_cnt','use_amt'])

for i in emd_cd_ary:
    tmp = monthly_output(i)
    data_result = data_result.append(tmp)

data_result['base_date'] = data_result.index
data_result

# 이제 mct_cat_cd 필요없으므로 삭제
Card_home=data_result.drop(['mct_cat_cd'],1)
Card_home

# 칼럼명 바꾸기
Card_home.rename(columns={'use_cnt':'use_cnt_home','use_amt':'use_amt_home'},inplace=True)
Card_home

,base_date,emd_cd,use_cnt_home,use_amt_home
2018-01-31,2018-01-31,50110250,241.00000,9544214.83226
2018-02-28,2018-02-28,50110250,248.57143,10986168.14286
2018-03-31,2018-03-31,50110250,253.71053,10104315.90789
2018-04-30,2018-04-30,50110250,268.58000,9984319.50667
2018-05-31,2018-05-31,50110250,287.87662,11574652.66234
...,...,...,...,...
2021-02-28,2021-02-28,50130620,74.91852,3094459.54074
2021-03-31,2021-03-31,50130620,73.89796,3175512.46939
2021-04-30,2021-04-30,50130620,74.99315,2940970.94521
2021-05-31,2021-05-31,50130620,80.43791,3820712.99346


In [141]:
# 행정동코드 매개변수로 받아서 해당 구역의 '외부 배출량'을 월별로 정리

pd.options.display.float_format = '{:.5f}'.format
def monthly_output1(emdcd):
    emd_data1 = Card_spend_out[Card_spend_out['emd_cd'] == emdcd]    # 해당 행정동 데이터만 추출
    emd_data_monthly1 = emd_data1.resample('M', on='base_date').mean()    # 월별로 합산
    emd_data_monthly1['emd_cd'] = emdcd
    return emd_data_monthly1

In [142]:
emd_cd_ary = [50110250, 50110253, 50110256, 50110259, 50110310, 50110320, 50110330, 50110510, 50110520, 50110530, 50110540, 50110550, 50110560, 50110570, 50110580, 50110590, 50110600, 50110610, 50110620, 50110630, 50110640, 50110650, 50110660, 50110670, 50110680, 50110690, 50130250, 50130253, 50130259, 50130310, 50130320, 50130510, 50130520, 50130530, 50130540, 50130550,
              50130560, 50130570, 50130580, 50130590, 50130600, 50130610, 50130620]

data_result1=pd.DataFrame(index=range(0,0), columns=['base_date','emd_cd','mct_cat_cd','use_cnt','use_amt'])

for i in emd_cd_ary:
    tmp1 = monthly_output1(i)
    data_result1 = data_result1.append(tmp1)

data_result1['base_date'] = data_result1.index
data_result1

# 이제 mct_cat_cd 필요없으므로 삭제
Card_out=data_result1.drop(['mct_cat_cd'],1)
# 칼럼명 바꾸기
Card_out.rename(columns={'use_cnt':'use_cnt_out','use_amt':'use_amt_out'},inplace=True)
Card_out

,base_date,emd_cd,use_cnt_out,use_amt_out
2018-01-31,2018-01-31,50110250,190.09211,6770950.83553
2018-02-28,2018-02-28,50110250,178.60432,6207769.64748
2018-03-31,2018-03-31,50110250,202.23129,6876453.89796
2018-04-30,2018-04-30,50110250,210.82432,7277611.03378
2018-05-31,2018-05-31,50110250,213.30128,7222843.14744
...,...,...,...,...
2021-02-28,2021-02-28,50130620,53.82609,1723436.00000
2021-03-31,2021-03-31,50130620,60.84962,1906192.95489
2021-04-30,2021-04-30,50130620,61.39844,1939595.03906
2021-05-31,2021-05-31,50130620,57.28369,1826894.56028


In [143]:
Card_spending_ver2=pd.merge(Card_home, Card_out, on=['base_date','emd_cd'])
Card_spending_ver2

,base_date,emd_cd,use_cnt_home,use_amt_home,use_cnt_out,use_amt_out
0,2018-01-31,50110250,241.00000,9544214.83226,190.09211,6770950.83553
1,2018-02-28,50110250,248.57143,10986168.14286,178.60432,6207769.64748
2,2018-03-31,50110250,253.71053,10104315.90789,202.23129,6876453.89796
3,2018-04-30,50110250,268.58000,9984319.50667,210.82432,7277611.03378
4,2018-05-31,50110250,287.87662,11574652.66234,213.30128,7222843.14744
...,...,...,...,...,...,...
1801,2021-02-28,50130620,74.91852,3094459.54074,53.82609,1723436.00000
1802,2021-03-31,50130620,73.89796,3175512.46939,60.84962,1906192.95489
1803,2021-04-30,50130620,74.99315,2940970.94521,61.39844,1939595.03906
1804,2021-05-31,50130620,80.43791,3820712.99346,57.28369,1826894.56028


In [144]:
Card_spending_ver2.to_csv("./데이터 전처리/Card_spending_ver2.csv",index_label=False)

In [145]:
pd.read_csv("./데이터 전처리/Card_spending_ver2.csv")

,base_date,emd_cd,use_cnt_home,use_amt_home,use_cnt_out,use_amt_out
0,2018-01-31,50110250,241.00000,9544214.83226,190.09211,6770950.83553
1,2018-02-28,50110250,248.57143,10986168.14286,178.60432,6207769.64748
2,2018-03-31,50110250,253.71053,10104315.90789,202.23129,6876453.89796
3,2018-04-30,50110250,268.58000,9984319.50667,210.82432,7277611.03378
4,2018-05-31,50110250,287.87662,11574652.66234,213.30128,7222843.14744
...,...,...,...,...,...,...
1801,2021-02-28,50130620,74.91852,3094459.54074,53.82609,1723436.00000
1802,2021-03-31,50130620,73.89796,3175512.46939,60.84962,1906192.95489
1803,2021-04-30,50130620,74.99315,2940970.94521,61.39844,1939595.03906
1804,2021-05-31,50130620,80.43791,3820712.99346,57.28369,1826894.56028
